In [ ]:
import torch
from torchvision import transforms
import timm
from PIL import Image
from tqdm import tqdm

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.7 MB/s eta 0:00:00


In [ ]:
model = timm.create_model('resnet50', pretrained=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [ ]:
model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [ ]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)

    return input_batch


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
def predict(model, input_tensor):
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)

    return output

image_path = '/content/drive/MyDrive/ondc/Catalog Digitization/ONDC Test Data _ Images/Product Images/7up_Cool_Drink_Pet.jpg'
input_batch = preprocess_image(image_path)
output = predict(model, input_batch)


In [ ]:
output.shape

torch.Size([1, 1000])

In [ ]:
import os
folderpath = "/content/drive/MyDrive/ondc/Catalog Digitization/ONDC Test Data _ Images/Product Images"
feat = {}
for i in tqdm(range(len(os.listdir(folderpath)))):
  imagepath = os.path.join(folderpath,os.listdir(folderpath)[i])
  inputbatch = preprocess_image(imagepath)
  output = predict(model,inputbatch)
  feat[os.listdir(folderpath)[i]]=output

100%|██████████| 1060/1060 [13:20<00:00,  1.32it/s]


In [ ]:
feat

In [ ]:
import pickle

my_dict = feat
file_path = '/content/featureswithnames.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(my_dict, file)

print(f'Dictionary saved to {file_path}')


Dictionary saved to /content/featureswithnames.pkl


In [ ]:

import pickle
with open('/content/featureswithnames.pkl', 'rb') as file:
    feat = pickle.load(file)

print(features)

In [ ]:
features = list(feat.values())

In [ ]:
from sklearn.neighbors import NearestNeighbors
KNN = NearestNeighbors(n_neighbors = 100,algorithm ="brute",metric = "manhattan")

In [ ]:
import numpy as np
features = np.array(features)

<ipython-input-32-511c63f26861>:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  features = np.array(features)
<ipython-input-32-511c63f26861>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.array(features)


In [ ]:
features = list(feat.values())

In [ ]:
for i in range(len(features)):
  features[i] = features[i].detach().numpy()
  features[i].squeeze()

In [ ]:
features = np.array(features)

In [ ]:
features.shape

(1060, 1, 1000)

In [ ]:
features = features.squeeze(1)

In [ ]:
features.shape

(1060, 1000)

In [ ]:
KNN.fit(features,)

NearestNeighbors(algorithm='brute', metric='manhattan', n_neighbors=100)

In [ ]:
processedimage = preprocess_image("/content/Screenshot 2024-02-06 002058.png")
output = predict(model, processedimage)
distances,indices = KNN.kneighbors(output.detach().numpy())

In [ ]:
distances

array([[394.48925781, 402.29251099, 415.71661377, 440.82125854,
        482.50997925, 483.1572876 , 483.59576416, 492.86801147,
        500.80776978, 509.52151489, 510.04495239, 511.10797119,
        511.16604614, 511.4630127 , 514.06414795, 518.6595459 ,
        522.46478271, 524.59655762, 525.65820312, 525.67297363,
        529.84075928, 532.65118408, 537.23913574, 537.53613281,
        538.78631592, 539.73284912, 542.00836182, 543.8637085 ,
        544.50366211, 545.18701172, 546.03741455, 547.04541016,
        547.62078857, 547.85235596, 549.81323242, 551.34344482,
        551.40087891, 551.86938477, 552.11315918, 553.49481201,
        553.65356445, 553.93920898, 554.36871338, 555.61846924,
        556.74932861, 558.56945801, 559.32800293, 559.59069824,
        559.79748535, 561.92987061, 562.77740479, 563.24835205,
        563.64941406, 563.69116211, 564.82739258, 566.53222656,
        568.51190186, 568.7588501 , 568.84515381, 569.12091064,
        570.02703857, 570.17077637, 570.

In [ ]:
indices

array([[ 102,   96,  274,  873, 1026, 1027,  108,  219,  706,  879,  501,
         213,  842,  204,  403,  845,    9,   62,  963, 1021,  713,  118,
         205,  951,  133,  208,  551,  827,  846,   82, 1016,  130,    3,
         901,  104,  478,  929,  968, 1028,  168,  531,  836,  603,  484,
         907, 1038,  278,  905,  914,   13,  957,  129,  100,  127,  924,
         216,   88,  911,  304,  917,   43,  828,  101,  708,   92,  535,
         119,   86, 1006,  289,  380,  923,  910,  965,  967,  899,  408,
         964,  629,   31,  512,  945,  128,   10,  156,  918,  927,  831,
         479,  124,   14,  295,  117, 1022, 1029,  812,  956,  302,  737,
         847]])

In [ ]:
names = list(feat.keys())

In [ ]:

for i in indices[0]:
  print(names[i],"\n")

Tide_Value_Pack_Detergent_Bar.png 

Tide_Detergent_Soap_Detergent_Soap.png 

Tide_With_Extra_Power_Lemon_Mint_Detergent_Powder.png 

Vim_Best_Ever_Dishwasher_Cleaning_Bar.png 

Cadbury_Bourn_Vita.png 

Challenge_Active_Wash_Extra_Lime_Active_Detergent_Powder.png 

Sunfeast_Bounce_Elaichi_Delight_Biscuits.png 

Rin_Smart_Foam_Detergent_Powder.png 

Surfexcel_Quick_Wash_Detergent_Powder.png 

Vim_Dishwash_Bar.png 

Pithambari_Shining_Powder.png 

Ruchi_Gold_Refined_Palmoleon_Oil.png 

Udhaiyam_Lamp_Oil.png 

Vim_Bar_Dish_Wash.png 

Horlicks_Chocolate_Delight_Flavour.png 

Udhaiyam_Moong_Dhall.png 

Haldiram_s_All_In_One_All.png 

Popi_Corn_Tomato_Chilli_Popcorn.png 

TRV_Dry_Black_Grapes_Seed.png 

Britannia_Nutri_Choice_Digestives_Biscuits.png 

Surya_Salt.png 

Kurkure_Green_Chutney_Style_Snacks.png 

Tide_With_Extra_Power_Jasmin_Rose_Detergent_Powder.png 

TRV_Dry_Figs_Afgan_Fig.png 

Kurkure_Green_Chutney_Style_Chips.png 

Yippee_Noodles_Mood_Masala.png 

Rajas_Puffed_Rice.png 

Too_

In [ ]:
import torch.onnx
import torch

example_input = torch.randn(1, 3, 224, 224)

torch.onnx.export(model, example_input, "model.onnx", verbose=True, input_names=["input"], output_names=["output"])


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1404: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message


In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 36.5 MB/s eta 0:00:00


In [ ]:
!python -m tf2onnx.convert --opset 13 --tflite --input "/content/model.onnx" --output "/content/model22.tflite"


2024-02-06 21:36:27.055699: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 21:36:27.055754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 21:36:27.057217: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 21:36:28.188746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
u

In [ ]:
!pip install tf2onnx.convert


ERROR: Could not find a version that satisfies the requirement tf2onnx.convert (from versions: none)
ERROR: No matching distribution found for tf2onnx.convert


In [ ]:
import onnx
onnx_model = onnx.load("/content/model.onnx")

In [ ]:
from onnx_tf.backend import prepare
tf_rep = prepare(onnx_model)